In [1]:

import warnings
warnings.filterwarnings("ignore")

import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable

import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_energy as me
import model_electricity as ml
import model_socioeconomic as se

import pathlib
import setup_analysis as sa
import sisepuede as ssp
import support_classes as sc
import support_functions as sf
import importlib
import time
import matplotlib.pyplot as plt
import sql_utilities as sq
from typing import *
import sqlalchemy
import sql_utilities as sqlutil
import time
importlib.reload(ma)
#importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(ml)




MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


<module 'model_electricity' from '/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_electricity.py'>

In [2]:
import model_attributes as ma
import model_variable as mv
import units_manager as um

In [308]:
#warnings.filterwarnings("default")

In [69]:
fp = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/out/sisepuede_summary_results_run_sisepuede_run_2024-02-10T17;45;35.578004/sisepuede_results_sisepuede_run_2024-02-10T17;45;35.578004_WIDE_INPUTS_OUTPUTS.csv"
df_test = pd.read_csv(fp)


# path = pathlib.Path(sa.dir_ref)
# path_fake_data = path.joinpath("fake_data", "fake_data_complete.csv")
# df_fake_data = pd.read_csv(path_fake_data)

def add_missing_lvst_elast(
    df: pd.DataFrame,
) -> pd.DataFrame:
    """
    add buffalo, goat, horse, and mule elasticities to a df
    """
    
    # sheep inputs
    vec_sheep = df["elasticity_lvst_sheep_demand_to_gdppc"].to_numpy()
    
    # buffalo
    vec_buffalo = df["elasticity_lvst_cattle_dairy_demand_to_gdppc"].to_numpy()
    vec_buffalo += df["elasticity_lvst_cattle_nondairy_demand_to_gdppc"].to_numpy()
    vec_buffalo /= 2


    flds_sheep = [
        "elasticity_lvst_goats_demand_to_gdppc",
        "elasticity_lvst_horses_demand_to_gdppc",
        "elasticity_lvst_mules_demand_to_gdppc"
    ]
    for fld in flds_sheep:
        if fld not in df.columns:
            df[fld] = vec_sheep
    
    fld_buffalo = "elasticity_lvst_buffalo_demand_to_gdppc"
    if fld_buffalo not in df.columns:
        df[fld_buffalo] = vec_buffalo

    
    return df

df_test = add_missing_lvst_elast(df_test);


df_base = (
    df_test[
        df_test["primary_id"].isin([125125])
    ]
    .reset_index(drop = True)
)

df_base_input = df_base[
    [x for x in df_base.columns if x not in sa.model_attributes.all_variable_fields_output]
]


    
    

In [74]:
import importlib
importlib.reload(sf)
importlib.reload(mv)
importlib.reload(um)
importlib.reload(ma)


fp = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/docs/source/csvs"
path = pathlib.Path(fp)
matt_new = ma.ModelAttributes(
    path, 
    fp_config = path.parents[2].joinpath("sisepuede.config")
)


# modvar = matt_new.dict_variables.get("Volumetric Energy Density")
# modvar.get_property("subsector")
# unit_energy = matt_new.get_unit("energy")
# unit_power = matt_new.get_unit("power")
# unit_power.get_unit_key("none")



modvar = ":math:\\text{CO}_2 Land Use Conversion Emission Factor"
#modvar = "Unadjusted Land Use Transition Probability"
#modvar = "GDP"
#modvar = "Population"
#modvar = "Land Use Yield Reallocation Factor"
modvar = ""
mov = matt_new.dict_variables.get(modvar)


#mov.build_fields(
#    # category_restrictions = ["croplands", "forests_primary"]#"cat_general": ["rural"]}
#    category_restrictions = "croplands"#{"cat_landuse_dim2": ["croplands", "forests_primary"]}
#)

matt_new.get_sector_list_from_projection_input(None)

MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


['AFOLU', 'Circular Economy', 'Energy', 'IPPU', 'Socioeconomic']

In [87]:
attr_time_period = matt_new.get_dimensional_attribute_table("time_period")
matt_new.attribute_group_key_other


'other'

In [82]:
attr = matt_new.get_dimensional_attribute_table(
    matt_new.dim_time_period
)

attr.table[[attr.key]]

,time_period
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [70]:
importlib.reload(mv)
importlib.reload(mafl)
importlib.reload(ma)
importlib.reload(mc)
importlib.reload(me)
importlib.reload(ml)
importlib.reload(se)

logger = None
matt_new = ma.ModelAttributes(
    path, 
    fp_config = path.parents[2].joinpath("sisepuede.config")
)




model_afolu = mafl.AFOLU(matt_new)
model_circecon = mc.CircularEconomy(matt_new);
model_ippu = mi.IPPU(matt_new);
model_elec = ml.ElectricEnergy(
    matt_new, 
    sa.dir_jl, 
    sa.dir_ref_nemo, 
    logger = None, 
    initialize_julia = True,
);

model_energy = me.NonElectricEnergy(matt_new);
model_socioeconomic = se.Socioeconomic(matt_new);


def run_strat(
    df_in: pd.DataFrame,
    project_nemomod: bool = False,
) -> pd.DataFrame:
    
    df_elec = None
    df_cs_integrated = df_in.copy()
    
    df_project_afolu = model_afolu.project(df_cs_integrated)
    df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_afolu], matt_new, "concatenate")
    
    # pass integrated df
    df_project_ce = model_circecon.project(df_cs_integrated)
    df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_ce], matt_new, "concatenate")
    
    # pass integrated df
    df_project_ip = model_ippu.project(df_cs_integrated)
    df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_ip], matt_new, "concatenate")
    
    # pass integrated df
    df_project_en = model_energy.project(df_cs_integrated);
    df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_en], matt_new, "concatenate")
    
    df_elec = None
    if project_nemomod:
        print("starting NemoMod")
        os.remove(sa.fp_sqlite_nemomod_db_tmp) if os.path.exists(sa.fp_sqlite_nemomod_db_tmp) else None
        engine = sqlalchemy.create_engine(f"sqlite:///{sa.fp_sqlite_nemomod_db_tmp}")
        t0 = time.time()
        df_elec = model_elec.project(df_cs_integrated, engine = engine, regions = "india")
        t1 = time.time()

    return df_project_afolu, df_cs_integrated, df_elec



dfap, dfc, dfe = run_strat(df_base_input, project_nemomod = True)





MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


  Activating project at `~/Documents/Projects/git_jbus/sisepuede/julia`


starting NemoMod
2024-29-Feb 11:40:05.742 Opened SQLite database at /Users/jsyme/Documents/Projects/git_jbus/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2024-29-Feb 11:40:05.773 Added NEMO structure to SQLite database at /Users/jsyme/Documents/Projects/git_jbus/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2024-29-Feb 11:40:07.821 Started modeling scenario.
NEMO encountered an error with the following message: SQLite.SQLiteException("attempt to write a readonly database").
To report this issue to the NEMO team, please submit an error report at https://leap.sei.org/support/. Please include in the report a list of steps to reproduce the error and the error message.


┌ Warning: Model period activity upper limits for technologies (TotalTechnologyModelPeriodActivityUpperLimit parameter) are not enforced when modeling selected years.
└ @ NemoMod ~/.julia/packages/NemoMod/d2ky8/src/scenario_calculation.jl:464
┌ Warning: Model period emission limits (ModelPeriodEmissionLimit parameter) are not enforced when modeling selected years.
└ @ NemoMod ~/.julia/packages/NemoMod/d2ky8/src/scenario_calculation.jl:6181


In [15]:
import sqlalchemy
import sql_utilities as sqlu

fp_sqlite_works = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/tmp/nemomod_intermediate_database_WORKS.sqlite"
fp_sqlite_new = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/tmp/nemomod_intermediate_database.sqlite"


# 

def get_tables_as_dict(
    fp: str,
) -> dict:
    

    engine = sqlalchemy.create_engine(f"sqlite:///{fp}")
    dict_out = {}
    
    tbl_names = engine.table_names()
    
    for tbl in tbl_names:
        tab = sqlcols_diff = []
check = []

for k, v_works in dict_works.items():
    
    v_new = dict_new.get(k)
    if v_new is None:
        print(f"{k} not found in New")
        continue
    
    if any(v_new.columns != v_works.columns):
        print(f"columns differ for {k}\n")
        cols_diff.append(k)
        continue
        
    
    if (v_new.shape != v_works.shape) & ("f" in v_new.columns):
        print(f"appending {k} to check\n")
        check.append(k)
        
    
    u.sql_table_to_df(
            engine,
            tbl
        )
        
        dict_out.update({tbl: tab})

    return dict_out, engine



dict_works, engine_works = get_tables_as_dict(fp_sqlite_works)
dict_new, engine_new = get_tables_as_dict(fp_sqlite_new)


cols_diff = []
check = []

for k, v_works in dict_works.items():
    
    v_new = dict_new.get(k)
    if v_new is None:
        print(f"{k} not found in New")
        continue
    
    if any(v_new.columns != v_works.columns):
        print(f"columns differ for {k}\n")
        cols_diff.append(k)
        continue
        
    
    if (v_new.shape != v_works.shape) & ("f" in v_new.columns):
        print(f"appending {k} to check\n")
        check.append(k)
        
    
dict_new.get("MinShareProduction")

In [66]:
dfe[[x for x in dfe.columns if x.startswith("emission_co2e_subsector")]].tail()

,emission_co2e_subsector_total_entc
51,689.425059
52,697.038743
53,695.387911
54,692.717155
55,690.684005


In [68]:
df_base[[x for x in dfe.columns if x.startswith("emission_co2e_subsector")]].tail()

In [ ]:
modvar = matt_new.dict_variables.get("Unadjusted Land Use Transition Probability")
modvar = matt_new.dict_variables.get(":math:\\text{CH}_4 Stationary Combustion Emission Factor")

#modvar.attribute("unit_mass")
modvar.attribute("unit_area")
modvar.name_fs_safe.upper()


In [5]:
importlib.reload(sf)
importlib.reload(mv)
import time


t0 = time.time()
#modvar = mv.ModelVariable(
#    attr_vars.table.iloc[2],
#    sa.model_attributes.dict_attributes,
#)
t1 = time.time()

"""
vs = mv.VariableSchema(
    modvar.dict_varinfo.get("variable_schema")
)
vs.mutable_elements_clean_ordered
vs.replace({"cat_industry": "cement"}, keys_are_clean = True,)
""";

#modvar.dict_varinfo
#modvar.dict_varinfo





series = attr_vars.table.iloc[2].copy()
series["variable_schema"] = "``ef_ippu_$UNIT-MASS$_$EMISSION-GAS$_per_$UNIT-MASS$_product_use_$CAT-INDUSTRY-DIM1$_$CAT-INDUSTRY-DIM2$_$CAT-TRANSPORTATION$`` (``$UNIT-MASS$ = tonne``, ``$EMISSION-GAS$ = co2``"
series["categories"] = "(``$CAT-INDUSTRY_DIM2$ = paper|cement|plastic``, ``$CAT-INDUSTRY-DIM1$ = product_use_lubricants|product_use_paraffin_wax|cement|plastic``, ``$CAT-TRANSPORTATION$ = aviation|rail_freight|dymm``)"
series["default_value"] = 0.0
#series["categories"] = "(``$CAT-INDUSTRY$ = all``, ``$CAT-TRANSPORTATION$ = aviation|rail_freight|dymm``)"

#series["variable_schema"] = "pij_$CAT-LANDUSE-DIM1$_to_$CAT-LANDUSE-DIM2$"
#series["categories"] = "all"

modvar = mv.ModelVariable(
    series,
    sa.model_attributes.dict_attributes,
)





# get_root_element("CAT-INDUSTRY-DIM4")
#modvar.schema.get_root_element("cat_industry_dim3")
# modvar.schema.mutable_elements_clean_ordered
modvar.dict_category_keys
modvar.schema.mutable_elements_clean_ordered





['cat_industry_dim1', 'cat_industry_dim2', 'cat_transportation']

In [1132]:
regex_at = re.compile("attribute_unit_(.*).csv")
regex_at.match("attribute_unit_temp_gr.csv").groups()



('temp_gr',)

In [1138]:
#sa.model_attributes.all_model_variables
fp = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede/docs/source/csvs/attribute_length.csv"
at = AttributeTable(fp, "length", clean_table_fields = True,)
at.table()

,length,unit_length,name,length_equivalent_m,length_equivalent_km,length_equivalent_mile,length_equivalent_mgm,length_equivalent_gm
0,m,``m``,Meters,1.000000e+00,0.00100,6.213730e-04,0.000001,1.000000e-09
1,km,``km``,Kilometers,1.000000e+03,1.00000,6.213700e-01,0.001000,1.000000e-06
2,Mile,``mile``,Miles,1.609340e+03,1.60934,1.000000e+00,0.001609,1.609340e-06
3,Mm,``mgm``,Megameters,1.000000e+06,1000.00000,6.213700e-04,1.000000,1.000000e-03
4,Gm,``gm``,Gigameters,1.000000e+09,1000000.00000,6.213700e-07,1000.000000,1.000000e+00


In [1052]:
modvar(
    df, 
    expand_to_all_categories = False,
    return_type = "array",
)

array([[0.92536768, 0.24845778, 0.37953433, 0.14847882, 0.25139435,
        0.71123234, 0.15856626, 0.48581205, 0.68147416, 0.54829414,
        0.08388734, 0.42894816, 0.66173826, 0.07984463, 0.47521661,
        0.38798217, 0.5266003 , 0.4420342 , 0.88720961, 0.88586851,
        0.70495647, 0.64484206, 0.15106266, 0.60722337],
       [0.92536768, 0.24845778, 0.37953433, 0.14847882, 0.25139435,
        0.71123234, 0.15856626, 0.48581205, 0.68147416, 0.54829414,
        0.08388734, 0.42894816, 0.66173826, 0.07984463, 0.47521661,
        0.38798217, 0.5266003 , 0.4420342 , 0.88720961, 0.88586851,
        0.70495647, 0.64484206, 0.15106266, 0.60722337],
       [0.92536768, 0.24845778, 0.37953433, 0.14847882, 0.25139435,
        0.71123234, 0.15856626, 0.48581205, 0.68147416, 0.54829414,
        0.08388734, 0.42894816, 0.66173826, 0.07984463, 0.47521661,
        0.38798217, 0.5266003 , 0.4420342 , 0.88720961, 0.88586851,
        0.70495647, 0.64484206, 0.15106266, 0.60722337],
       [0.925

In [1026]:
#modvar.name.replace(":math:", "").replace("{", "").replace("\\text", "").replace("}")
df = np.full((20, len(modvar.fields_space)), -1)
df = pd.DataFrame(df, columns = modvar.fields_space)
for fld in modvar.fields:
    df[fld] = np.random.rand()
    
    


In [ ]:
importlib.reload(mv)

series = attr_vars.table.iloc[2].copy()
series["variable_schema"] = "``ef_ippu_$UNIT-MASS$_$EMISSION-GAS$_per_$UNIT-MASS$_product_use_$CAT-INDUSTRY-DIM1$_$CAT-INDUSTRY-DIM2$_$CAT-TRANSPORTATION$`` (``$UNIT-MASS$ = tonne``, ``$EMISSION-GAS$ = co2``"
series["categories"] = "(``$CAT-INDUSTRY_DIM2$ = paper|cement|plastic``, ``$CAT-INDUSTRY-DIM1$ = product_use_lubricants|product_use_paraffin_wax|cement|plastic``, ``$CAT-TRANSPORTATION$ = aviation|rail_freight|dymm``)"

modvar = mv.ModelVariable(
    series,
    sa.model_attributes.dict_attributes,
)




modvar.get_categories_by_element(
    #{
    #    "cat_industry_dim1": ["metals", "plastic", "chemicals"],#"``metals|plastic|chemicals``"
    #},
    dict_category_space = modvar.dict_category_key_space,
)




In [816]:
modvar.schema.get_root_element("cat_indusdtry_dim2")
20000/122.2

163.66612111292963

In [785]:
regex.match("CAT-INDUSTRY-DIM3")
mv.clean_element("-")





'_'

In [666]:
modvar.schema.mutable_elements_ordered
modvar.schema.space_char
re.compile(f".*{modvar.schema.space_char}{modvar.schema.flag_dim}(\d*)$").match("ahitheX-DIM34")





<re.Match object; span=(0, 13), match='ahitheX-DIM34'>

In [527]:
#modvar.schema.get_attribute("emission_gas")
hasattr(attr_cat, "key_values")
#modvar.get_categories_from_string(modvar.dict_varinfo.get(modvar.key_categories), "|", "`")

modvar.get_categories_by_element(
    category_subspace = {
        "cat_industry": "``product_use_lubricants|product_use_paraffin_wax``",
        "cat_transportation": "``product_use_lubricants``|``product_use_paraffin_wax``",
        "cat_transportation": "``product_use_lubricants``|``product_use_paraffin_wax``",
    },
    dict_category_space = modvar.dict_category_keys,
    
)

{'cat_industry': ['product_use_lubricants', 'product_use_paraffin_wax']}

In [390]:
modvar.schema.get_mutable_element(
    "ef_ippu_tonne_co2_per_tonne_production_thisun_jfijdf_clinker",
    "cat_industry"
) 

'thisun_jfijdf'

In [290]:
modvar.build_field_from_elements(
    {"cat_industry": "cement"}, 
    keys_are_clean = True
)



'ef_ippu_tonne_co2_per_tonne_production_cement_clinker'

In [302]:
class Field:
    
    def __init__(self,
        field: str,
        cat: Union[Dict, str, None],
    ):
        
        self.field = field
        self.cat = cat
        
        
    
    def __call__(self,
    ):
        
        return self.field
    

field = Field("ef_ippu_tonne_co2_per_tonne_production_cement_clinker", {"cat_industry": "cement"})

field(), field.cat
        

('ef_ippu_tonne_co2_per_tonne_production_cement_clinker',
 {'cat_industry': 'cement'})

In [278]:
sr = "``ef_ippu_$UNIT-MASS$_$EMISSION-GAS$_per_$UNIT-MASS$_production_$CAT-INDUSTRY-I$_$CAT-TRANSPORTATION-J$`` (``$UNIT-MASS$ = tonne``, ``$EMISSION-GAS$ = ch4``)"
vs2 = mv.VariableSchema(sr)
vs2.mutable_elements_ordered


vs2.replace({"cat_industry_i": "cementitude", "cat_transportation_j": "airplanification"}, keys_are_clean = True,)

'ef_ippu_tonne_ch4_per_tonne_production_cementitude_airplanification'

In [ ]:
vs.mutable_elements_clean_ordered



In [ ]:
sa.model_attributes.dict_attributes.get("abbreviation_subsector").table;

In [244]:
dict1 = {}

dict1[None] = 2
dict1

{None: 2}

In [195]:
def get_variable_schema_base(#self,
    var_schema_raw: str,
) -> str:
    """
    Using a raw variable schema, replace any unit and emission specifications
    """
    
    dict_replacements, schema  = mv.clean_schema(
        var_schema_raw, 
        return_type = "all"
    )
    
    return dict_replacements, schema

dict_replacements, schema = get_variable_schema_base(
    modvar.dict_varinfo.get("variable_schema")
)

In [198]:
isinstance(pd.DataFrame, type)

True

In [201]:
#regex = re.compile("(.*)\$(.*)\$(.*$)")
regex = re.compile("(?<=\$)(.*?)(?=\$)")
regex.findall("this $THIS$ $THIS2$")
#regex = re.search("(?<=\$)(.*?)(?=\$)", "this $HERE$ $NOPE$")





['THIS', ' ', 'THIS2']

In [182]:
string = "this $THIS$ $THIS2$"
string.count("$")

0

In [193]:
modvar.dict_varinfo.get("variable_schema")

'``ef_ippu_$UNIT-MASS$_$EMISSION-GAS$_per_$UNIT-MASS$_production_$CAT-INDUSTRY$`` (``$UNIT-MASS$ = tonne``, ``$EMISSION-GAS$ = ch4``)'

In [215]:
arr_1 = np.array([7, 11, 13, 17, 19, 23, 29])

prs = [2, 3, 5, 7, 11, 13, 17, 19]
arr_2 = []
gaps = []
for i in range(np.prod(prs)):
    check = [i%x for x in prs]
    binary = int(0 in check)
    
    arr_2.append(binary)
    if not binary:
        gaps.append(i)


P = np.prod(prs)
gaps_g1 = [x for x in gaps if x > 1]

classes = []
if False:
    l = []
    for i in range(1, 2):#len(arr_1) + 1):
        subs = list(itertools.combinations(gaps_g1, i))

        l.extend(subs)


    classes = []
    for i, sub in enumerate(l):
        val = np.prod(sub)
        mod = val%P

        classes.append(mod)

    classes = set(classes)
diffs = np.array(gaps[1:]) - np.array(gaps[0:-1])
len([x for x in diffs if x == 4])
w = np.abs(np.array(diffs[1:]) - np.array(diffs[0:-1]))
w = np.where(w == 0)
w;




pi1 = prs[-1]
counts_2 = []

P_prev = np.prod(prs[0:-1])
# check counts of 2 among each subdivision--should be relatively smooth
for i in range(pi1):
    gps = [x for x in gaps if (x >= i*P_prev) & (x < (i + 1)*P_prev)]
    
    ds = np.array(gps[1:]) - np.array(gps[0:-1])
    n = len([x for x in ds if x == 2])
    
    print(n)
    counts_2.append(n)
    #len([x for x in diffs[0:n] if x == 2])
    
    
import sympy
# non-prime nums
gaps_nopr = [x for x in gaps if (not sympy.isprime(x)) and (x > 1)]


l = []
for i in range(1, len(gaps_nopr) + 1):
    subs = list(itertools.combinations(gaps_nopr, i))
    
    l.extend(subs)

classes = []
for i, sub in enumerate(l):
    val = np.prod(sub)
    mod = val%P
    
    classes.append(mod)
    
    
gaps_nopr = [x for x in gaps if (not sympy.isprime(x)) and (x > 1)]
classes = []
for i in range(1, 3):#len(gaps_nopr) + 1):
    combs = itertools.combinations(gaps_nopr, i)
    
    for comb in combs:
        class_check = np.prod(comb)%P
        classes.append(class_check) if (class_check not in classes) else None
    
classes2 = classes.copy()



